In [2]:
import pandas as pd
df = pd.read_csv(r"C:\Users\Logan\Desktop\School Files\GIS 5572\Final\AllTrafficCost.csv")

print(df.head())

   SourceOID      Date       Sport  TrafficCost  BaseCost  SegmentLength  \
0   35232607  12032013  Basketball     0.115790  0.335916       0.169147   
1   35232950   1062015  Basketball     0.109809  0.098828       0.127014   
2   35232950   1132015  Basketball     0.109809  0.098828       0.127014   
3   35232950   1232018  Basketball     0.109809  0.098828       0.127014   
4   35232950   1292013  Basketball     0.109809  0.098828       0.127014   

    Latitude  Longitude Timestamp  Weekday?  Evening?  delta_cost  
0  44.903296 -93.274734  083000PM      True      True   -0.220126  
1  44.905975 -93.274550  080000PM      True      True    0.010981  
2  44.905975 -93.274550  080000PM      True      True    0.010981  
3  44.905975 -93.274550  080000PM      True      True    0.010981  
4  44.905975 -93.274550  080000PM      True      True    0.010981  


In [11]:
import os
# Load the data from the CSV file into a DataFrame
data = pd.read_csv(r"C:\Users\Logan\Desktop\School Files\GIS 5572\Final\AllTrafficCost.csv")

# Create a random sample of 20% of the data
sampled_data = data.sample(frac=0.2, random_state=42)

# Save the sampled data to a new CSV file
sampled_data.to_csv('SampledData.csv', index=False)

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

# Load the data from the CSV file into a DataFrame
data = pd.read_csv('SampledData.csv')

# Filter out rows where 'Sport' column equals 'line'
data_filtered = data[data['Sport'] != 'line']

# Select the categorical columns of interest
categorical_columns = ['Sport', 'Timestamp', 'Weekday?']

# Perform one-hot encoding for all categorical columns
encoder = OneHotEncoder()
encoded_columns = pd.DataFrame(encoder.fit_transform(data_filtered[categorical_columns]).toarray(), index=data_filtered.index)
encoded_columns.columns = encoder.get_feature_names_out(categorical_columns)

# Select the numerical columns
numerical_columns = data_filtered[['Latitude', 'Longitude']]

# Concatenate the encoded categorical columns with the numerical ones
X = pd.concat([numerical_columns, encoded_columns], axis=1)

# Assuming 'delta_cost' is the column you want to predict
y = data_filtered['delta_cost']

# Split the dataset into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Linear Regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

# Print the coefficients and intercept of the model
print("\nModel Coefficients:")
for feature, coef in zip(X.columns, model.coef_):
    print(f"{feature}: {coef}")
print("Intercept:", model.intercept_)

Mean Squared Error: 0.03234145967621752
R-squared Score: 0.0019411758250607036

Model Coefficients:
Latitude: -0.24231410478886153
Longitude: -0.17266544693263958
Sport_Basketball: 1362253677.9308362
Sport_Football: 1362253677.9303617
Sport_Hockey: 1362253677.9288197
Sport_Volleyball: 1362253677.931258
Timestamp_010000PM: -1508196784.0506644
Timestamp_011500PM: -1508196784.048714
Timestamp_013000PM: -1508196784.0533144
Timestamp_020000PM: -1508196784.0515232
Timestamp_023000PM: -1508196784.0505505
Timestamp_030000PM: -1508196784.0522096
Timestamp_033000PM: -1508196784.054928
Timestamp_040000PM: -1508196784.0521257
Timestamp_041500PM: -1508196784.0550663
Timestamp_043000PM: -1508196784.0533917
Timestamp_050000PM: -1508196784.0522125
Timestamp_053000PM: -1508196784.0506907
Timestamp_060000PM: -1508196784.0516832
Timestamp_063000PM: -1508196784.0538316
Timestamp_070000PM: -1508196784.0525186
Timestamp_071500PM: -1508196784.0527132
Timestamp_073000PM: -1508196784.0536067
Timestamp_080000PM

In [28]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

# Load the data from the CSV file into a DataFrame
data = pd.read_csv('SampledData.csv')

# Filter out rows where 'Sport' column equals 'line'
data_filtered = data[data['Sport'] != 'line']

# Define the categorical and numerical columns
categorical_columns = ['Sport', 'Timestamp', 'Weekday?']
numerical_columns = ['Latitude', 'Longitude']

# Perform one-hot encoding for all categorical columns
encoder = OneHotEncoder()
encoded_columns = encoder.fit_transform(data_filtered[categorical_columns])
encoded_columns_df = pd.DataFrame(encoded_columns.toarray(), index=data_filtered.index)
encoded_columns_df.columns = encoder.get_feature_names_out(categorical_columns)

# Concatenate the encoded categorical columns with the numerical ones
X = pd.concat([data_filtered[numerical_columns], encoded_columns_df], axis=1)

# Assuming 'delta_cost' is the column you want to predict
y = data_filtered['delta_cost']

# Initialize the Linear Regression model
model = LinearRegression()

# Fit the model on the full data
model.fit(X, y)

# Suppose you have new data in the form of a DataFrame called 'new_data'
new_data = pd.DataFrame({
    'Sport': ['Football'],  # Example new categorical value
    'Timestamp': ['070000PM'],  # Example new timestamp value
    'Weekday?': [False],  # Example new weekday value
    'Latitude': [44.903296],  # Example latitude
    'Longitude': [-93.274734]  # Example longitude
})

# Perform one-hot encoding for the new categorical columns
encoded_new_data = encoder.transform(new_data[categorical_columns])
encoded_new_data_df = pd.DataFrame(encoded_new_data.toarray(), columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate the encoded categorical columns with the numerical ones for the new data
encoded_new_data_final = pd.concat([new_data[numerical_columns].reset_index(drop=True), encoded_new_data_df.reset_index(drop=True)], axis=1)

# Ensure the column order in the new data matches the training data
encoded_new_data_final = encoded_new_data_final.reindex(columns=X.columns, fill_value=0)

# Now you can use the trained model to make predictions on the new data
new_predictions = model.predict(encoded_new_data_final)

print("Predicted delta_cost for the new data:", new_predictions)

Predicted delta_cost for the new data: [0.04968357]
